In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Tensorflow imports

# for building model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Flatten, GlobalMaxPooling1D, Dense, Dropout

# for Padding
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences

# for Tokenization 
from tensorflow.keras.preprocessing.text import Tokenizer

# NLTK imports
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# For visualization
import matplotlib.pyplot as plt
# import seaborn as sns

# For regular expressions
import re

# For data preprocessing
from string import punctuation, digits

In [6]:
# Read in data
train_df = pd.read_csv('dataset/Corona_NLP_train.csv', encoding="Latin-1")
test_df = pd.read_csv('dataset/Corona_NLP_test.csv', encoding="Latin-1")

print(f"train dataset shape >> {train_df.shape}")
print(f"test dataset shape >> {test_df.shape}")

train dataset shape >> (41157, 6)
test dataset shape >> (3798, 6)


In [15]:
# Creat new dataset (which will consist from 2 columns: label, data)

def data_label_split(dataset):
    data = dataset['OriginalTweet']
    label = dataset['Sentiment']
    return data,label

train_data,train_label = data_label_split(train_df)
test_data,test_label = data_label_split(test_df)

train = pd.DataFrame({
    'label':train_label,
    'data':train_data
})

test = pd.DataFrame({
    'label':test_label,
    'data':test_data
})

# Define function which will make new labels

def reassign_label(x):
    if x == "Extremely Positive" or x == "Positive":
        return 1
    elif x =="Extremely Negative" or x =="Negative":
        return -1
    elif x =="Neutral":
        return 0

train.label = train.label.apply(lambda x:reassign_label(x))
test.label = test.label.apply(lambda x:reassign_label(x))

train

,label,data
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...
1,1,advice Talk to your neighbours family to excha...
2,1,Coronavirus Australia: Woolworths to give elde...
3,1,My food stock is not the only one which is emp...
4,-1,"Me, ready to go at supermarket during the #COV..."
...,...,...
41152,0,Airline pilots offering to stock supermarket s...
41153,-1,Response to complaint not provided citing COVI...
41154,1,You know itÂs getting tough when @KameronWild...
41155,0,Is it wrong that the smell of hand sanitizer i...


# Data preprocessing

In [12]:
def remove_punctuation(s):
    list_punctuation = list(punctuation)
    for i in list_punctuation:
        s = s.replace(i,'')
    return s.lower()

def clean_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', '', sentence) # remove URL adresses
    sentence = re.sub(r"\@(\w+)", '', sentence) # remove usernames
    sentence = re.sub(r"\#(\w+)", '', sentence) # remove hashtags
    sentence = re.sub(r"\$(\w+)", '', sentence) # remove cashtags
    sentence = sentence.replace("-",' ')
    tokens = sentence.split()
    tokens = [remove_punctuation(w) for w in tokens] # remove punctuations
    stop_words = set(stopwords.words('english')) # remove stopwords
    tokens = [w for w in tokens if not w in stop_words]
    remove_digits = str.maketrans('', '', digits)
    tokens = [w.translate(remove_digits) for w in tokens]
    tokens = [w.strip() for w in tokens]
    tokens = [w for w in tokens if w!=""]
    tokens = ' '.join(tokens)
    return tokens

def process_text(text):
  text = str(text) #Convert string to str
  #Lowers the string
  text = text.lower()
  #Removes the full url
  url_remove = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
  text = re.sub(url_remove,' ',text)
  #Removes the punctuation
  text = ''.join([string for string in text if string not in punctuation and not string.isdigit()])
  #Removes any more special characters
  special_character = re.compile(r'[^a-zA-Z]')
  text = re.sub(special_character,' ', text)
  text = text.strip() #Strip white spaces
  text = text.split(' ')
  text = ' '.join([string for string in text if string not in stopwords.words('english')])#Removing all stop words
  return text

In [16]:
# Clean sentences in train and test data
# train.data = train.data.apply(lambda sentence:clean_sentence(sentence))
# test.data = test.data.apply(lambda sentence:clean_sentence(sentence))

train.data = train.data.apply(process_text)
test.data = test.data.apply(process_text)

In [19]:
train

,label,data
0,0,menyrbie philgahan chrisitv
1,1,advice talk neighbours family exchange phone n...
2,1,coronavirus australia woolworths give elderly ...
3,1,food stock one empty please dont panic en...
4,-1,ready go supermarket covid outbreak im pa...
...,...,...
41152,0,airline pilots offering stock supermarket shel...
41153,-1,response complaint provided citing covid relat...
41154,1,know getting tough kameronwilds rationing to...
41155,0,wrong smell hand sanitizer starting turn ...


In [20]:
# Splitting data to train and test

# Train data
train_data = train.data
train_label = train.label

# Test data
test_data = test.data
test_label = test.label

In [22]:
# Convert categorical variable (in our case: -1, 0, 1) into dummy/indicator variables. Such as -1 to 1 0 0 
train_label = pd.get_dummies(train_label)
test_label = pd.get_dummies(test_label)

,-1,0,1
0,0,1,0
1,0,0,1
2,0,0,1
3,0,0,1
4,1,0,0
...,...,...,...
41152,0,1,0
41153,1,0,0
41154,0,0,1
41155,0,1,0
